In [ ]:
#Imports and paths

import os
import subprocess

# Paths
working_directory="/Users/katherineridley/APPFIRE_image"
Lifs = os.path.join(working_directory, "Lifs")
OMETIFFs = os.path.join(working_directory, "OMETIFFs")
MaxPTIFFs = os.path.join(working_directory, "MaxProjections")


#download bftools onto your machine, point to bfconvert here:
bfconvert_path = "/Users/katherineridley/Downloads/bftools/bfconvert" 



In [ ]:
#convert Lifs to Tiffs
# Works with TWO channel images, splits them into separate stacks, saves as c0 and c1

# Create the output directory if it doesn't exist
if not os.path.exists(OMETIFFs):
    os.makedirs(OMETIFFs)

# Iterate through all .lif files in the input directory
for filename in os.listdir(Lifs):
    if filename.endswith(".lif"):
        input_file = os.path.join(Lifs, filename)
        

        # Loop through each series and channel
        
        for channel in [0, 1]:
                output_file = os.path.join(OMETIFFs, "%n_c{}.ome.tiff".format(channel))
                command = f"{bfconvert_path} -noflat -channel {channel} {input_file} {output_file}"
                subprocess.run(command, shell=True)

In [ ]:
#Create Max Projections, create merged file for Max Projections for TWO CHANNELS ONLY

if not os.path.exists(MaxPTIFFs):
    os.makedirs(MaxPTIFFs)

processed_files = {}  # To keep track of processed files for merging

# Loop through all .ome.tiff files in the input directory
for filename in os.listdir(OMETIFFs):
    if filename.endswith(".ome.tiff"):
        # Splitting filename to get base name and channel
        parts = filename.split('_')
        channel = parts[-1] 
        c = channel.split('.')[0]
        #print(c, 'here is channel') # Assuming channel is the second last element
        base_name = '_'.join(parts[:-1])
        
        image_path = os.path.join(OMETIFFs, filename)
        output_path = os.path.join(MaxPTIFFs, f"{base_name}_{c}_maxp.ome.tiff")

        process_image(image_path, output_path)

        # Store the output paths for merging
        if base_name not in processed_files:
            processed_files[base_name] = {}
        processed_files[base_name][c] = output_path
            
#print(processed_files)
# Merge channels for each base image
for base_name, channels in processed_files.items():
    #print('both together at merge', base_name, channels)
    if 'c0' in channels and 'c1' in channels:
        
        merge_channels(channels['c0'], channels['c1'], 
                       os.path.join(MaxPTIFFs, f"{base_name}_maxp_merge.ome.tiff"))

In [ ]:
pip install aicsimageio scikit-image napari

In [ ]:
from aicsimageio import AICSImage
from skimage.filters import threshold_otsu
from skimage.measure import label
from skimage.segmentation import clear_border
import numpy as np
import napari
from scipy import ndimage as ndi
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)


# Read OME-TIFF file and metadata
#image = io.imread('/Users/katherineridley/APPFIRE_image/OMETIFFs_2/F_2_M04_OC_L23_P+_231123_1.ome.tiff')
'''print(img)
image = img.data  # Get the image data as a numpy array
metadata = img.metadata  # Get the metadata
print(image)'''

#print(image.shape)

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('/Users/katherineridley/APPFIRE_image/TIFFs/C_1_M04_OC_L23_P+_151123_1.tiff')
assert img is not None, "file could not be read, check with os.path.exists()"
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)

unknown = cv.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv.imshow('Original Image', img)
cv.waitKey(0)


cv.imshow('Grayscale Image', gray)
cv.waitKey(0)

cv.imshow('Thresholded Image', thresh)
cv.waitKey(0)







In [8]:
import numpy as np
import h5py
import skimage.io as io
from skimage.filters import threshold_otsu
from scipy.ndimage import label, binary_fill_holes
import matplotlib.pyplot as plt
import os
from skimage.segmentation import mark_boundaries
from skimage.transform import resize
from matplotlib.colors import ListedColormap
from skimage import io, exposure, data
from skimage.filters import gaussian
from skimage.filters import threshold_local
from skimage.filters import try_all_threshold
from skimage.filters import threshold_yen, sobel, threshold_isodata, threshold_multiotsu
import napari
import pandas as pd
from skimage.segmentation import find_boundaries
from napari.utils import nbscreenshot
import imageio
metadata_dict = {}

input_dir = "/Users/katherineridley/APPFIRE_image/TIFFs/"
output_dir = "/Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/"
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

plaques = [image for image in os.listdir(input_dir) if ('P+' in image) & (image.endswith('.tif'))]


# Initialize a dictionary to store masks for each channel
fl_channels = [0,1]
channel_masks = {}

for image in plaques:

    
    # Initialize an entry for each image
    '''channel_masks[image] = {channel: [] for channel in fl_channels}
    channel_masks[image]['masks'] = []
    channel_masks[image]['raw']=[]
    channel_masks[image]['background_corrected']=[]
    channel_masks[image]['central_masks']=[]
    channel_masks[image]['binary_masks']=[]'''

    print(image)

    
    # Construct the control image filename by replacing 'P+' with 'P-'
    control_image = image.replace('P+', 'P-')

    control_image_path = os.path.join(input_dir, control_image)

    if not os.path.exists(control_image_path):
        continue



    # Read the experimental image stack
    image_stack = io.imread(os.path.join(input_dir, image))

    # Read the corresponding negative control image stack
    control_stack = io.imread(os.path.join(input_dir, control_image))

    # Ensure the dimensions match between the image and control stacks
    if image_stack.shape != control_stack.shape:
        print(f"Mismatch in image and control stack dimensions for {image}. Skipping.")
        continue

    outputfolder = os.path.join(output_dir, image)
    os.makedirs(outputfolder, exist_ok=True) 
    print('output', outputfolder)

    rawfolder = os.path.join(outputfolder,'raw_image_stacks')
    os.makedirs(rawfolder, exist_ok=True) 

    bgcorrectedfolder = os.path.join(outputfolder,'background_corrected_stacks')
    os.makedirs(bgcorrectedfolder, exist_ok=True) 

    scaledfolder = os.path.join(outputfolder,'min_max_scaled_stacks')
    os.makedirs(scaledfolder, exist_ok=True) 

    negfolder = os.path.join(outputfolder,'negative_control_stacks')
    os.makedirs(negfolder, exist_ok=True) 

    fullmaskfolder = os.path.join(outputfolder,'full_mask_stacks')
    os.makedirs(fullmaskfolder, exist_ok=True) 

    maskwindowfolder = os.path.join(outputfolder,'mask_window_stacks')
    os.makedirs(maskwindowfolder, exist_ok=True) 

    imagewindowfolder = os.path.join(outputfolder,'image_window_stacks')
    os.makedirs(imagewindowfolder, exist_ok=True) 

    binaryfolder = os.path.join(outputfolder,'binary_mask_stacks')
    os.makedirs(binaryfolder, exist_ok=True) 

    threshsurface = os.path.join(outputfolder,'surface_boundaries')
    os.makedirs(threshsurface, exist_ok=True) 



    # Define fluorescent channels


    # Initialize a dictionary to store thresholds

    for channel in fl_channels:
        if channel == 0:
            protein = 'Methoxy-0-4'

        else:
            protein = 'OC'

        binary_masks = []
        # Process each z-plane
        for z_plane in range(image_stack.shape[0]):
            # Calculate the central region
            height, width = image_stack[z_plane, channel].shape
            center_y, center_x = height // 2, width // 2
            delta_y, delta_x = height // 4, width // 4  # 50% of the height and width
            y_min, y_max = center_y - delta_y, center_y + delta_y
            x_min, x_max = center_x - delta_x, center_x + delta_x

            center_fraction = 0.4  # For example, use 40% of the height and width instead of 50%
            delta_y, delta_x = int(height * center_fraction / 2), int(width * center_fraction / 2)
            y_min, y_max = center_y - delta_y, center_y + delta_y
            x_min, x_max = center_x - delta_x, center_x + delta_x

            # Crop the smaller central region
            central_region = image_stack[z_plane, channel][y_min:y_max, x_min:x_max]

            central_control = control_stack[z_plane, channel][y_min:y_max, x_min:x_max]

            backgroundthreshold = threshold_otsu(central_control)
            originalthreshold = threshold_otsu(central_region)
            #print('whole image:', np.mean(image_stack), 'central z plane:', np.mean(central_region), 'whole control:', np.mean(control_stack), 'control z plane:', np.mean(central_control))

            background_corrected_image = (central_region.astype(np.float32) - (central_control.astype(np.float32)))
            
            # Set any negative values to zero
            background_corrected_image[background_corrected_image < 0] = 0

            hard_cutoff= 5
            image_cutoff = image_stack > hard_cutoff
            v_min, v_max = np.percentile(background_corrected_image, (0.05, 99.99))
            # Apply threshold to the central region
            image_minmax_scaled = exposure.rescale_intensity(background_corrected_image, in_range=(v_min, v_max), out_range=(0,50))
            
            #print('mean:', np.mean(image_minmax_scaled))
            #print('max:',np.max(image_minmax_scaled))
            #sigma=1
            #blurred_image = gaussian(image_minmax_scaled, sigma=sigma, mode='nearest')
            
            #better_contrast = exposure.rescale_intensity(image_minmax_scaled[z_plane, channel], in_range=(v_min, v_max))
            threshold_value = (threshold_otsu(image_minmax_scaled))

            #threshold_value = threshold_local(central_region, block_size=3, offset=-0.1)

            #apply thrshold from entire stack>>
            '''image_minmax_scaled = exposure.rescale_intensity(image_stack)
            sigma=0.1
            #blurred_image = gaussian(image_stack, sigma=sigma, mode='nearest')
            v_min, v_max = np.percentile(image_stack, (0.2, 99.8))

            

            better_contrast = exposure.rescale_intensity(image_minmax_scaled)
            threshold_value = threshold_otsu(image_stack[:, channel])
            intensity_cutoff = np.percentile(image_stack, 20) 
            if channel == 0:
                offset = 0.3
            else:
                offset = 1
            threshold_value = threshold_value * offset'''
            
            #binary_mask = central_region > threshold_value

            #>>new code<<
            #block_size = 35  # This is a parameter you may need to tune
            #local_thresh = threshold_local(central_region, block_size, offset=50)
            #binary_mask = central_region > local_thresh

            binary_mask = central_region > threshold_value

            #print(threshold_value)


            
            # Process the binary mask
            filled_mask = binary_fill_holes(binary_mask)
            cleaned_mask, _ = label(filled_mask)
            size = np.bincount(cleaned_mask.flatten())

            #print(threshold_value)
            
            binary_cleaned_mask = (cleaned_mask > 0).astype(int)
            #print('sum', np.sum(binary_cleaned_mask), 'size:', binary_cleaned_mask.size)

            print((np.sum(binary_cleaned_mask) / binary_cleaned_mask.size) * 100)
            # Check the condition for empty mask
            if ((threshold_value < 4.5) & (np.sum(binary_cleaned_mask) > 0.3 * binary_cleaned_mask.size)) | ((np.sum(binary_cleaned_mask) > 0.65 * binary_cleaned_mask.size)):
                
                #unique_values = np.unique(cleaned_mask)
                #print(unique_values)
                #if threshold_value < 4.5 and np.sum(cleaned_mask) > 0.9 * cleaned_mask.size:
                cleaned_mask = np.zeros_like(cleaned_mask)
            

            # Check if there are non-zero elements in 'size' after the first element
            if len(size) > 1:
                largest_component = np.argmax(size[1:]) + 1
                cleaned_mask = cleaned_mask == largest_component
            else:
                # If no connected components, use an empty mask
                cleaned_mask = np.zeros_like(cleaned_mask)



            # Create an empty mask for the whole image
            full_cleaned_mask = np.zeros_like(image_stack[z_plane, channel])
            # Place the processed central region back into the full mask
            full_cleaned_mask[y_min:y_max, x_min:x_max] = cleaned_mask

            #print(full_cleaned_mask)

            # Instead of binary_masks.append, use:
            '''channel_masks[image][channel].append(full_cleaned_mask)
            channel_masks[image]['masks'].append(full_cleaned_mask)
            
            channel_masks[image]['raw'].append(image_stack)
            channel_masks[image][channel].append(image_stack)
            channel_masks[image]['background_corrected'].append(background_corrected_image)
            channel_masks[image][channel].append(background_corrected_image)
            channel_masks[image]['central_masks'].append(cleaned_mask)
            channel_masks[image][channel].append(cleaned_mask)
            channel_masks[image]['binary_masks'].append(binary_cleaned_mask)
            channel_masks[image][channel].append(binary_cleaned_mask)

            #save channel_masks

            masks = {'z':z_plane,
                     'channel':channel,
                'raw':image_stack[z_plane, channel],
                'controlraw':control_stack[z_plane, channel],
                'backgroundcorrected':image_minmax_scaled,
                'thresholdmask':full_cleaned_mask,
                'binarymask':binary_cleaned_mask,
                'thresholdwindow':cleaned_mask,
                'rawwindow':central_region}
            
            


            channel_masks[image] = masks

            io.imsave(rawfolder+'/raw_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), image_stack[z_plane, channel], check_contrast=False)
            io.imsave(negfolder+'/control_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), control_stack[z_plane, channel],check_contrast=False)
            io.imsave(bgcorrectedfolder+'/bgcorr_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), background_corrected_image,check_contrast=False)
            io.imsave(scaledfolder+'/scaled_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), image_minmax_scaled,check_contrast=False)
            
            io.imsave(maskwindowfolder+'/mask_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), cleaned_mask,check_contrast=False)
            io.imsave(imagewindowfolder+'/window_{}_z{}_{}.tif'.format(image[:3], z_plane, protein), central_region,check_contrast=False)
            io.imsave(binaryfolder+'/binary_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), binary_cleaned_mask,check_contrast=False)'''

            stacked_boundaries = find_boundaries(cleaned_mask)

            io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
        

            #io.imsave(fullmaskfolder+'/mask_{}_z{}_{}.tiff'.format(image[:3], z_plane, protein), full_cleaned_mask, check_contrast=False)

            '''parts = image.split('_')
            metadata = {
                'channel': channel,
                'slide': parts[0],
                'section': parts[1],
                'region': parts[4],
                'plaque': parts[5],
                'date': parts[6],
                'number': parts[7].split('.')[0]  # Assuming the file extension follows
            }

            # Add additional data
            metadata['binarymasksize'] =np.sum(binary_cleaned_mask)
            metadata['zplane'] = z_plane
            metadata['znumber'] = image_stack.shape[0]
            metadata['otsuthreshold'] = threshold_value
            metadata['mean'] = np.mean(image_minmax_scaled)
            

            # Add the metadata to the dictionary with the image name as key
            metadata_dict[image] = metadata

            # After processing all images, add them to the viewer




            #brightness_increase = 1000  # Adjust this value as needed

            # Increase brightness
            #brightened_image = image_stack + brightness_increase

            image_minmax_scaled = exposure.rescale_intensity(background_corrected_image)
            better_contrast = exposure.rescale_intensity(image_minmax_scaled)
            v_min, v_max = np.percentile(image_stack, (0.2, 99.8))

            
            print('making plots')
            better_contrast = exposure.rescale_intensity(image_stack, in_range=(v_min, v_max))

            central_contrast = exposure.rescale_intensity(central_region, in_range=(v_min, v_max))
            #fig, ax = try_all_threshold(z_plane, figsize=(10, 8), verbose=False)
            #plt.show()

            # Visualize the results
            plt.figure(figsize=(10, 5))
            plt.suptitle(f'Protein {protein}, Z-Plane: {z_plane}')

            # Display the original image for the current z-plane
            plt.subplot(1, 2, 1)
            plt.imshow(better_contrast[z_plane, channel], cmap='gray')
            plt.title('Original Image')

            # Display the segmented volume for the current z-plane
            plt.subplot(1, 2, 2)
            plt.imshow(full_cleaned_mask, cmap='gray')
            plt.title('Segmented Volume')

            plt.tight_layout()
            plt.savefig(outputfolder+f"/{image}_{z_plane}_c{channel}_thresh_otsu.png")
            plt.close()

            boundaries = find_boundaries(cleaned_mask)

            # Visualize in Matplotlib
            plt.figure(figsize=(10, 5))
            plt.imshow(cleaned_mask, cmap='gray')  # Show the original mask
            plt.imshow(boundaries, cmap='Purples', alpha=0.5)  # Overlay boundaries
            plt.title(f'Channel {channel} - z {z_plane} with Boundaries')
            plt.savefig(outputfolder+f"/{image}_{z_plane}_c{channel}_boundaries.png")
            plt.close()



#maskdf = pd.DataFrame.from_dict(channel_masks, orient='index')
#masks.to_csv(os.path.join(input_dir, 'masks.csv'))
df=pd.DataFrame.from_dict(metadata_dict, orient='index')
df.to_csv(os.path.join(input_dir, 'metadata.csv'))


viewer = napari.Viewer()

for image, channel_info in channel_masks.items():
    # 'channel_info' is the nested dictionary for each image
    for channel, masks in channel_info.items():
        if channel == 'masks':
            # This is the list of all masks, handle accordingly
            # ... process 'mask_list' which contains all masks for this image ...
            
            stacked_masks = np.stack(masks, axis=0)
            # Stack boundaries similarly
            stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

            # Add original mask
            viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

            # Add boundaries as another layer
            viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')

            frames = []

# Iterate through the z-stacks (slices) of the binary mask
            for i in range(stacked_masks.shape[0]):  # Assuming the first dimension is the z-axis
                frame = stacked_masks[i, :, :]  # Get the i-th slice of the mask
                print('making gif')
                # You might want to convert the binary mask to a suitable format for GIF
                # For example, multiplying by 255 for a typical black-and-white representation
                frame_image = (frame * 255).astype(np.uint8)
                
                # Append the frame to the list
                frames.append(frame_image)

            # Save the frames as a GIF
            
            imageio.mimsave(outputfolder+'c{channel}_gif.gif', frames, format='GIF', fps=10)  # Adjust fps as needed

            print("GIF created at:", output_path)

            #napari.run()

    
 

            video_path = os.path.join(outputfolder, '3dvideo.mp4')

            # Record a video
            with napari.gui_qt():
                # Set the desired duration and frames per second for the video
                total_duration_sec = 5
                fps = 30
                ('Creating video')
                total_frames = total_duration_sec * fps
                for frame in range(total_frames):
                    # Here you can modify the viewer, e.g., rotate or zoom
                    # Example: Rotate the view
                    viewer.dims.ndisplay = 3
                    viewer.dims.point = [frame, frame, frame]  # Change this as needed
                    # Capture the frame
                    nbscreenshot(viewer, canvas_only=False).save(f'{frame}.png')

                # Use imageio to compile the saved frames into a video
                import imageio
                with imageio.get_writer(video_path, fps=fps) as writer:
                    for frame in range(total_frames):
                        writer.append_data(imageio.imread(f'{frame}.png'))




for channel in fl_channels:
    binary_masks = []
# Process each z-plane
    for z_plane in range(image_stack.shape[0]):
    # Create binary masks for each channel
    
    
        threshold_value = thresholds[channel][z_plane]
        binary_mask = image_stack[z_plane, channel] > threshold_value
        binary_masks.append(binary_mask)

        # Stack the binary masks to form a binary stack
        binary_stack = np.stack(binary_masks)

        # Fill holes in the binary mask
        filled_stack = binary_fill_holes(binary_stack)

        # Remove small structures or noise
        cleaned_stack, _ = label(filled_stack)

        # Select only the largest connected component (if needed)
        size = np.bincount(cleaned_stack.flatten())
        cleaned_stack = cleaned_stack == np.argmax(size[1:]) + 1

        # Visualize the results
        plt.figure(figsize=(10, 5))
        plt.suptitle(f'Methoxy-0-4, Z-Plane: {z_plane}')

        # Display the original image for the current z-plane
        plt.subplot(1, 2, 1)
        plt.imshow(image_stack[z_plane, channel], cmap='gray')
        plt.title('Original Image')

        # Display the segmented volume for the current z-plane
        plt.subplot(1, 2, 2)
        plt.imshow(cleaned_stack[z_plane], cmap='gray')
        plt.title('Segmented Volume')

        plt.tight_layout()
        plt.show()

image_stack = io.imread('/Users/katherineridley/APPFIRE_image/TIFFs/F_3_M04_OC_L1_P+_231123_1.tif')
thresholds = {}
# Assume your fluorescent channel is at index 1
fl_channels = [0,1]
#image_stack.shape[1]
# Loop over each z-stack
print(list(range(image_stack.shape[0])))
for channel in fl_channels:
    for z_plane in list(range(image_stack.shape[0])):
        
        # Initialize an empty list to store the threshold values
        

        # Apply Otsu's method to determine the threshold for each channel
        
        threshold = threshold_otsu(image_stack[z_plane, channel])
        # Store the threshold in the nested dictionary
        thresholds[channel][z_plane] = threshold


    # Create a binary mask for each channel using the respective threshold
    binary_stack = np.stack([image_stack[z_plane, channel] > threshold for channel, threshold in zip(fl_channels, thresholds)])

    # Fill holes in the binary mask
    filled_stack = binary_fill_holes(binary_stack)

    # Remove small structures or noise
    cleaned_stack, _ = label(filled_stack)

    # Select only the largest connected component (if needed)
    size = np.bincount(cleaned_stack.flatten())
    cleaned_stack = cleaned_stack == np.argmax(size[1:]) + 1


#z_plane = 15
print(thresholds)
# Display the original image
for z_plane in list(range(image_stack.shape[0])):
    plt.figure(figsize=(10, 5))  # Adjust the figure size as needed
    plt.suptitle(f'Methoxy-0-4, Z-Plane: {z_plane}')

    # Display the original image for the current z-plane
    plt.subplot(1, 2, 1)
    plt.imshow(image_stack[z_plane, fl_channels[0]], cmap='gray')
    plt.title('Original Image')

    # Display the segmented volume for the current z-plane
    plt.subplot(1, 2, 2)
    plt.imshow(cleaned_stack[z_plane], cmap='gray')
    plt.title('Segmented Volume')

    # Adjust the layout and show the plot
    plt.tight_layout()
    plt.show()
# Apply Otsu's method to determine the threshold
threshold1 = threshold_otsu(image_stack[:, fl_channel[0]])
print(threshold1)

# Applying the threshold to create a binary mask
binary_stack1 = image_stack[:, fl_channel[0]] > (threshold1)

# Fill holes in the binary mask
filled_stack1 = binary_fill_holes(binary_stack1)

# Remove small structures or noise
cleaned_stack1, _ = label(filled_stack1)

# Select only the largest connected component (if needed)
size = np.bincount(cleaned_stack1.flatten())
cleaned_stack1 = cleaned_stack1 == np.argmax(size[1:]) + 1

# Apply Otsu's method to determine the threshold
threshold2 = threshold_otsu(image_stack[:, fl_channel[1]])


# Applying the threshold to create a binary mask
binary_stack2 = image_stack[:, fl_channel[1]] > (threshold2)

# Fill holes in the binary mask
filled_stack2 = binary_fill_holes(binary_stack2)

# Remove small structures or noise
cleaned_stack2, _ = label(filled_stack2)

# Select only the largest connected component (if needed)
size = np.bincount(cleaned_stack2.flatten())
cleaned_stack2 = cleaned_stack2 == np.argmax(size[1:]) + 1

#z_plane = 15

for z_plane in range(image_stack.shape[0]):

# Display the original image
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(image_stack[z_plane, fl_channel[1]], cmap='gray')
    plt.title('Original Image')
    plt.suptitle('OC')
    # Display the segmented volume
    plt.subplot(1, 2, 2)
    plt.imshow(cleaned_stack2[z_plane], cmap='gray')
    plt.title('Segmented Volume')

    # Adjust the layout and show the plot
    plt.tight_layout()
    plt.show()

import os
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.transform import resize
from matplotlib.colors import ListedColormap

output_dir = "/Users/katherineridley/APPFIRE_image/TIFFs/F_3_M04_OC_L1_P+_231123_1"
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

# Channel colors for visualization
colors_rgb = [(0, 1, 0), (1, 0, 1)]

# Create a custom colormap
cmap = ListedColormap(colors_rgb)
# Create a merged segmented volume
merged_stack = np.logical_or(cleaned_stack[..., 0], cleaned_stack[..., 1])

# Loop over each z-plane
for z_plane in range(image_stack.shape[0]):
    # Create the figure and axes
    fig, ax = plt.subplots()

    # Display the original image
    ax.imshow(image_stack[z_plane, 0], cmap='gray')

    # Display the segmented volume as outlines
    for channel in range(image_stack.shape[1]):
        ax =plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image_stack[z_plane, channel], cmap='gray')
        plt.title('Original Image')

        # Display the segmented volume
        plt.subplot(1, 2, 2)
        plt.imshow(cleaned_stack[z_plane], cmap='gray')
        plt.title('Segmented Volume')
        #ax.imshow(mark_boundaries(image_stack[z_plane, channel], cleaned_stack[z_plane, channel], color=colors_rgb[channel]), cmap=cmap)

    # Set the axes labels and title
        
        #plt.title(f'Z-Plane {z_plane}, {channel}')

        # Save the figure
        plt.savefig(os.path.join(output_dir, f"z_plane_{z_plane}_{channel}_segmentation.png"))
        plt.close(fig)  # Close the figure to release memory



# Create the figure and axes for the stacked plot
fig, axs = plt.subplots(cleaned_stack.shape[0], 2, sharex='all', sharey='all', figsize=(10, 10))

# Loop over each z-plane
for z_plane in range(cleaned_stack.shape[0]):
    # Display the original image in the first column
    axs[z_plane, 0].imshow(image_stack[z_plane, 0], cmap='gray')
    axs[z_plane, 0].set_title(f'Z-Plane {z_plane}')

    # Display the segmented volume in the second column
    axs[z_plane, 1].imshow(cleaned_stack[z_plane])
    axs[z_plane, 1].set_title('Segmented Volume')

# Set the axes labels
fig.text(0.5, 0.04, 'X', ha='center')
fig.text(0.04, 0.5, 'Y', va='center', rotation='vertical')

# Adjust the layout and save the figure
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "segmentation_stacked.png"))
plt.close(fig)  # Close the figure to release memory


# Determine the bounding box of the segmented volume
bbox = np.argwhere(merged_stack)
(min_y, min_x), (max_y, max_x) = bbox.min(0), bbox.max(0)

# Add padding
padding = 50
min_x, max_x = max(0, min_x - padding), min(max_x + padding, image_stack.shape[3])
min_y, max_y = max(0, min_y - padding), min(max_y + padding, image_stack.shape[2])

# Resize the image and segmented volume
resized_image = resize(image_stack[:, :, min_y:max_y, min_x:max_x], (image_stack.shape[0], 2, 512, 512))
resized_segmentation = resize(merged_stack[:, min_y:max_y, min_x:max_x], (image_stack.shape[0], 512, 512))

io.imsave(os.path.join(output_dir, "resized_image.tif"), resized_image, plugin='tifffile')
np.save(os.path.join(output_dir, "resized_segmentation.npy"), resized_segmentation)'''

<tifffile.TiffPages @8> invalid page offset 46579112


C_1_M04_OC_DGG_P+_151123_1.tif
output /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/C_1_M04_OC_DGG_P+_151123_1.tif
0.6253604382929643
0.8103854286812765
1.1443915801614764
1.370266243752403
1.506632064590542
1.715686274509804
2.775374855824683
2.5507016532103037
2.4732074202229914
2.4593906189926953


/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/C_1_M04_OC_DGG_P+_151123_1.tif/surface_boundaries/surface_C_1_z0_Methoxy-0-4.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/C_1_M04_OC_DGG_P+_151123_1.tif/surface_boundaries/surface_C_1_z1_Methoxy-0-4.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/

2.313413110342176
2.385500768935025
2.4978373702422147
2.2443291041906956
2.241325451749327
2.1013552479815454
1.6940599769319493
1.2429113802383698
0.9857987312572088
0.7455065359477124
0.588715878508266
0.6758217993079585


/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/C_1_M04_OC_DGG_P+_151123_1.tif/surface_boundaries/surface_C_1_z12_Methoxy-0-4.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/C_1_M04_OC_DGG_P+_151123_1.tif/surface_boundaries/surface_C_1_z13_Methoxy-0-4.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c

...

/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z15_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z16_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt346

16.03950403690888
31.013312187620144
25.451148596693578
36.98817762399077
24.45033160322953
24.885260476739717
27.977821030372933


/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z22_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z23_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt346

13.578311226451364
13.53565936178393
13.28815840061515
9.036788735101883
12.21765667051134
11.085880430603614
2.445573817762399


/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z29_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)
/var/folders/c7/rvxz8xg96pxfhyt34649yfqh0000gn/T/ipykernel_14401/1682557767.py:267: UserWarning: /Users/katherineridley/APPFIRE_image/TIFFs/ProcessedImages/B_4_M04_OC_L4_P+_131123_1.tif/surface_boundaries/surface_B_4_z30_OC.npy is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  io.imsave(threshsurface+'/surface_{}_z{}_{}.npy'.format(image[:3], z_plane, protein), stacked_boundaries,check_contrast=False)


In [ ]:

df=pd.DataFrame.from_dict(metadata_dict, orient='index')
df.to_csv(os.path.join(input_dir, 'metadata.csv'))

In [3]:
with h5py.File('channel_masks.h5', 'w') as hdf:
    for key, value in channel_masks.items():
        group = hdf.create_group(key)
                
        for inner_key, inner_value in value.items():
            print(inner_value)
            if isinstance(inner_value, np.ndarray):
                group.create_dataset(inner_key, data=inner_value)
            else:
                # Store small metadata as attributes
                group.attrs[inner_key] = inner_value

21
1
[[0 1 2 ... 0 2 2]
 [2 1 3 ... 0 0 0]
 [0 0 4 ... 0 1 3]
 ...
 [0 0 1 ... 0 0 0]
 [2 0 5 ... 0 0 0]
 [0 1 0 ... 0 1 0]]
[[0 0 0 ... 1 3 0]
 [0 0 0 ... 4 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 3 0 2]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
[[ 0.        0.        0.891884 ...  0.        0.        1.783768]
 [ 0.        0.891884  0.       ...  0.        1.783768 15.162028]
 [ 0.891884  0.        2.675652 ...  0.        0.891884  0.      ]
 ...
 [ 0.        0.891884  0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.891884]
 [ 0.891884  0.        0.       ...  0.891884  0.        0.891884]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False .

In [7]:
with h5py.File('channel_masks.h5', 'r') as hdf:
    print(hdf)
    for group_name in hdf:
        print(group_name)
        group = hdf[group_name]
        for dataset_name in group:
            dataset = group[dataset_name]
            data = dataset[...]  # Read the data
            print(f"Data in {dataset_name}: {data}")


<HDF5 file "channel_masks.h5" (mode r)>
A_4_M04_OC_P+_CA1_011123_1.tif
Data in backgroundcorrected: [[ 0.4142248  1.2426745  2.485349  ...  0.         1.2426745  0.       ]
 [ 1.2426745  1.6568992  2.071124  ...  0.         0.4142248  0.4142248]
 [ 0.4142248  0.8284496  0.4142248 ...  0.         0.         0.       ]
 ...
 [ 1.2426745  0.         0.        ... 14.912094  10.769846   7.456047 ]
 [ 0.         0.         2.8995738 ... 18.225891  10.769846   2.071124 ]
 [ 0.         0.         0.        ...  7.0418215  3.7280235  4.970698 ]]
Data in binarymask: [[1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]]
Data in controlraw: [[0 3 5 ... 3 4 2]
 [0 7 2 ... 5 4 5]
 [3 5 3 ... 5 6 1]
 ...
 [2 7 1 ... 6 1 0]
 [3 1 1 ... 4 6 3]
 [1 0 1 ... 5 4 3]]
Data in raw: [[ 1  4  8 ... 14 17 20]
 [ 8  3  4 ...  6 11 15]
 [ 4  3  7 ...  8 12 12]
 ...
 [ 5  9  4 ...  6  4  9]
 [12 10  5 ...  5  5  4]
 [23  4 12 ... 11  8  8]]
Data in

In [4]:


channels = pd.DataFrame.from_dict(channel_masks, orient='index')
channels.to_csv('channel_masks_2.csv')



In [ ]:
from naparimovie import Movie
from napari_animation import Animation
import vispy

viewer = napari.Viewer()

#red = vispy.color.Colormap([[0., 0.0, 0.0], [
green = vispy.color.Colormap([[0.8, 1.0, 0.8], [0.0, 1.0, 0.0]])
blue = vispy.color.Colormap([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])

for image, channel_info in channel_masks.items():
    if image == 'B_3_M04_OC_L23_P+_061123_1.tif':
        print(image)
        # 'channel_info' is the nested dictionary for each image
        for channel, masks in channel_info.items():
            print(channel)
            if channel == 0:
                #label = Colormap(['#EE82EE'])
                # This is the list of all masks, handle accordingly
                # ... process 'mask_list' which contains all masks for this image ...
                
                stacked_masks = np.stack(masks, axis=0)
                # Stack boundaries similarly
                stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

                # Add original mask
                #viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

                # Add boundaries as another layer
                labels=viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')
                #labels.colormap = 'blue', blue

            elif channel== 1:

                stacked_masks = np.stack(masks, axis=0)
                # Stack boundaries similarly
                stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

                # Add original mask
                #viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

                # Add boundaries as another layer
                labels = viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')
                #labels=viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')
                labels.blending='additive'
                #labels.colormap = 'red', green

                frames = []

            #movie = Movie(viewer)
            #movie.inter_steps = 30
            #movie.make_movie(name = 'movie.mp4', resolution = 300, fps = 20)
            #movie.make_gif('gifmovie.gif')
            
            napari.run()

  


In [ ]:
pip install git+https://github.com/guiwitz/naparimovie.git@master#egg=naparimovie

In [ ]:
for channel, masks in channel_masks.items():
    stacked_masks = np.stack(masks, axis=0)
    # Stack boundaries similarly
    stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

    # Add original mask
    viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

    # Add boundaries as another layer
    viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')

    napari.run()

In [ ]:
viewer = napari.Viewer()

for channel, masks in channel_masks.items():
    print(channel)
    stacked_masks = np.stack(masks, axis=0)
    # Stack boundaries similarly
    stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

    # Add original mask
    viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

    # Add boundaries as another layer
    viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')

    napari.run()

for channel, masks in channel_masks.items():
    for i, mask in enumerate(masks):
        # Find boundaries
        boundaries = find_boundaries(mask)

        # Visualize in Matplotlib
        plt.figure(figsize=(10, 5))
        plt.imshow(mask, cmap='gray')  # Show the original mask
        plt.imshow(boundaries, cmap='magma', alpha=0.5)  # Overlay boundaries
        plt.title(f'Channel {channel} - Mask {i} with Boundaries')
        plt.show()

video_path = os.path.join(outputfolder, '3dvideo.mp4')

# Record a video
with napari.gui_qt():
    # Set the desired duration and frames per second for the video
    total_duration_sec = 5
    fps = 30

    total_frames = total_duration_sec * fps
    for frame in range(total_frames):
        # Here you can modify the viewer, e.g., rotate or zoom
        # Example: Rotate the view
        viewer.dims.ndisplay = 3
        viewer.dims.point = [frame, frame, frame]  # Change this as needed
        # Capture the frame
        nbscreenshot(viewer, canvas_only=False).save(f'{frame}.png')

    # Use imageio to compile the saved frames into a video
    import imageio
    with imageio.get_writer(video_path, fps=fps) as writer:
        for frame in range(total_frames):
            writer.append_data(imageio.imread(f'{frame}.png'))




In [ ]:

import numpy as np
import skimage.io as io
import skimage.io as io
from skimage.filters import threshold_otsu
from scipy.ndimage import label, binary_fill_holes
import matplotlib.pyplot as plt
import os
from skimage.segmentation import mark_boundaries
from skimage.transform import resize
from matplotlib.colors import ListedColormap
from skimage import io, exposure, data
from skimage.filters import gaussian
from skimage.filters import threshold_local
from skimage.filters import try_all_threshold
from skimage.filters import threshold_yen, sobel, threshold_isodata, threshold_multiotsu
import napari
import pandas as pd
from skimage.segmentation import find_boundaries



input_dir = "/Users/katherineridley/APPFIRE_image/TIFFs/"
output_dir = "/Users/katherineridley/APPFIRE_image/TIFFs/thresholded/"
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

plaques = [image for image in os.listdir(input_dir) if 'P+' in image]
viewer = napari.Viewer()

# Initialize a dictionary to store masks for each channel
fl_channels = [0, 1]
channel_masks = {channel: [] for channel in fl_channels}



#for image in plaques:

image = 'F_3_M04_OC_L1_P+_231123_1.tif'
# Construct the control image filename by replacing 'P+' with 'P-'
control_image = image.replace('P+', 'P-')

# Read the experimental image stack
image_stack = io.imread(os.path.join(input_dir, image))

# Read the corresponding negative control image stack
control_stack = io.imread(os.path.join(input_dir, control_image))

# Ensure the dimensions match between the image and control stacks
assert image_stack.shape == control_stack.shape, "Mismatch in image and control stack dimensions"




outputfolder = os.path.join(output_dir, image)
os.makedirs(outputfolder, exist_ok=True) 
# Define fluorescent channels


# Initialize a dictionary to store thresholds

for channel in fl_channels:
    if channel == 0:
        protein = 'Methoxy-0-4'

    else:
        protein = 'OC'

    binary_masks = []
    # Process each z-plane
    for z_plane in range(image_stack.shape[0]):
        # Calculate the central region
        height, width = image_stack[z_plane, channel].shape
        center_y, center_x = height // 2, width // 2
        delta_y, delta_x = height // 4, width // 4  # 50% of the height and width
        y_min, y_max = center_y - delta_y, center_y + delta_y
        x_min, x_max = center_x - delta_x, center_x + delta_x

        center_fraction = 0.4  # For example, use 40% of the height and width instead of 50%
        delta_y, delta_x = int(height * center_fraction / 2), int(width * center_fraction / 2)
        y_min, y_max = center_y - delta_y, center_y + delta_y
        x_min, x_max = center_x - delta_x, center_x + delta_x

        # Crop the smaller central region
        central_region = image_stack[z_plane, channel][y_min:y_max, x_min:x_max]

        central_control = control_stack[z_plane, channel][y_min:y_max, x_min:x_max]

        backgroundthreshold = threshold_otsu(central_control)
        originalthreshold = threshold_otsu(central_region)
        #print('whole image:', np.mean(image_stack), 'central z plane:', np.mean(central_region), 'whole control:', np.mean(control_stack), 'control z plane:', np.mean(central_control))

        background_corrected_image = (central_region.astype(np.float32) - (central_control.astype(np.float32)))
        
        # Set any negative values to zero
        background_corrected_image[background_corrected_image < 0] = 0

        hard_cutoff= 5
        image_cutoff = image_stack > hard_cutoff
        v_min, v_max = np.percentile(background_corrected_image, (0.05, 99.99))
        # Apply threshold to the central region
        image_minmax_scaled = exposure.rescale_intensity(background_corrected_image, in_range=(v_min, v_max), out_range=(0,50))
        
        #print('mean:', np.mean(image_minmax_scaled))
        #print('max:',np.max(image_minmax_scaled))
        #sigma=1
        #blurred_image = gaussian(image_minmax_scaled, sigma=sigma, mode='nearest')
        
        #better_contrast = exposure.rescale_intensity(image_minmax_scaled[z_plane, channel], in_range=(v_min, v_max))
        threshold_value = (threshold_otsu(image_minmax_scaled))

        #threshold_value = threshold_local(central_region, block_size=3, offset=-0.1)

        #apply thrshold from entire stack>>
        '''image_minmax_scaled = exposure.rescale_intensity(image_stack)
        sigma=0.1
        #blurred_image = gaussian(image_stack, sigma=sigma, mode='nearest')
        v_min, v_max = np.percentile(image_stack, (0.2, 99.8))

        

        better_contrast = exposure.rescale_intensity(image_minmax_scaled)
        threshold_value = threshold_otsu(image_stack[:, channel])
        intensity_cutoff = np.percentile(image_stack, 20) 
        if channel == 0:
            offset = 0.3
        else:
            offset = 1
        threshold_value = threshold_value * offset'''
        
        #binary_mask = central_region > threshold_value

        #>>new code<<
        #block_size = 35  # This is a parameter you may need to tune
        #local_thresh = threshold_local(central_region, block_size, offset=50)
        #binary_mask = central_region > local_thresh

        binary_mask = central_region > threshold_value

        #print(threshold_value)


        # Process the binary mask
        filled_mask = binary_fill_holes(binary_mask)
        cleaned_mask, _ = label(filled_mask)
        size = np.bincount(cleaned_mask.flatten())

        # Check if there are non-zero elements in 'size' after the first element
        if len(size) > 1:
            largest_component = np.argmax(size[1:]) + 1
            cleaned_mask = cleaned_mask == largest_component
        else:
            # If no connected components, use an empty mask
            cleaned_mask = np.zeros_like(cleaned_mask)

        # Create an empty mask for the whole image
        full_cleaned_mask = np.zeros_like(image_stack[z_plane, channel])
        # Place the processed central region back into the full mask
        full_cleaned_mask[y_min:y_max, x_min:x_max] = cleaned_mask

        #print(full_cleaned_mask)

        # Instead of binary_masks.append, use:
        channel_masks[channel].append(full_cleaned_mask)
        


        # After processing all images, add them to the viewer




        #brightness_increase = 1000  # Adjust this value as needed

        # Increase brightness
        #brightened_image = image_stack + brightness_increase

        '''image_minmax_scaled = exposure.rescale_intensity(background_corrected_image)
        better_contrast = exposure.rescale_intensity(image_minmax_scaled)
        v_min, v_max = np.percentile(image_stack, (0.2, 99.8))

        

        better_contrast = exposure.rescale_intensity(image_stack, in_range=(v_min, v_max))
        #fig, ax = try_all_threshold(z_plane, figsize=(10, 8), verbose=False)
        #plt.show()

        # Visualize the results
        plt.figure(figsize=(10, 5))
        plt.suptitle(f'Protein {protein}, Z-Plane: {z_plane}')

        # Display the original image for the current z-plane
        plt.subplot(1, 2, 1)
        plt.imshow(better_contrast[z_plane, channel], cmap='gray')
        plt.title('Original Image')

        # Display the segmented volume for the current z-plane
        plt.subplot(1, 2, 2)
        plt.imshow(full_cleaned_mask, cmap='gray')
        plt.title('Segmented Volume')

        plt.tight_layout()
        plt.savefig(os.path.join(outputfolder, f"{image}_{z_plane}_c{channel}_thresh_otsu.png"))
        plt.show()'''




for channel, masks in channel_masks.items():
    stacked_masks = np.stack(masks, axis=0)
    # Stack boundaries similarly
    stacked_boundaries = np.stack([find_boundaries(mask) for mask in masks], axis=0)

    # Add original mask
    viewer.add_image(stacked_masks, name=f'Channel {channel} Masks', contrast_limits=(0, 1))

    # Add boundaries as another layer
    viewer.add_labels(stacked_boundaries, name=f'Channel {channel} Boundaries')

napari.run()




In [ ]:
import tifffile as tiff
import cv2 as cv
import numpy as np

# Read the OME-TIFF file
img_stack = tiff.imread('/Users/katherineridley/APPFIRE_image/OMETIFFs/F_2_M04_OC_L23_P+_231123_1_c0.ome.tiff', is_ome=False)

# Check if the image stack has been loaded
assert img_stack is not None, "file could not be read, check with os.path.exists()"

# Process each slice in the Z-stack
for z in range(img_stack.shape[0]):  # Assuming Z is the first dimension
    img = img_stack[z, :, :]

    # Convert to grayscale if necessary
    if len(img.shape) > 2:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    else:
        gray = img

    # Apply your existing processing steps to each slice
    # For example, thresholding:
    ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)

    unknown = cv.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    #markers = cv.watershed(img,markers)
    #img[markers == -1] = [255,0,0]
    # ... continue with your processing steps

    # Display the processed slice
    cv.imshow(f'Slice {z}', gray)
    cv.waitKey(0)

cv.destroyAllWindows()


### java dependencies need sorting before being able to deploy javabridge

## skeleton notebook

In [ ]:
import os
import javabridge
import bioformats
import numpy as np
from tifffile import imsave

def lif_to_tiff(input_path, output_dir, ome_tiff=False):
    # Start the Java Virtual Machine
    javabridge.start_vm(class_path=bioformats.JARS)

    # Read the .lif file with bioformats
    r = bioformats.ImageReader(input_path)
    
    # Determine number of series (individual images) in the .lif file
    n_series = r.rdr.getSeriesCount()

    for series in range(n_series):
        r.rdr.setSeries(series)
        
        # Get image dimensions
        x, y, z, c, t = (r.rdr.getSizeX(), r.rdr.getSizeY(), r.rdr.getSizeZ(), 
                         r.rdr.getSizeC(), r.rdr.getSizeT())

        # Initialize an empty array for image data
        img_data = np.empty((t, z, y, x, c), dtype=np.uint16)

        # Fill the array with image data
        for ti in range(t):
            for zi in range(z):
                for ci in range(c):
                    img_data[ti, zi, :, :, ci] = r.read(z=zi, t=ti, c=ci, rescale=False)

        # Save as TIFF or OME-TIFF
        filename = f"image_series_{series}"
        filepath = os.path.join(output_dir, filename + ('.ome.tiff' if ome_tiff else '.tiff'))
        imsave(filepath, img_data)

    # Close the Java Virtual Machine
    javabridge.kill_vm()

if __name__ == "__main__":
    input_lif = "path_to_your_file.lif"
    output_directory = "path_for_output_files"
    lif_to_tiff(input_lif, output_directory, ome_tiff=True)


In [ ]:
##concepts: NOT EXECUTABLE

import numpy as np
import javabridge
import bioformats

from skimage import morphology, measure, filters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Initialize Java Virtual Machine
javabridge.start_vm(class_path=bioformats.JARS)

def load_ome_tiff(file_path):
    # Read OME-TIFF using bioformats
    img, meta = bioformats.read_image(file_path, rescale=False, wants_max_intensity=True, return_metadata=True)
    return img, meta

def preprocess(data):
    # Preprocessing steps, e.g., normalization
    return (data - data.min()) / (data.max() - data.min())

def segment(data):
    # Segmentation using a simple threshold (this can be replaced with more complex methods)
    threshold = filters.threshold_otsu(data)
    return data > threshold

def morphological_analysis(data):
    convex_hull = morphology.convex_hull_image(data)
    complexity = measure.perimeter(data) / np.sqrt(4 * np.pi * measure.area(data))
    return convex_hull, complexity

def extract_features(data):
    # Feature extraction for ML
    features = {
        'mean_intensity': np.mean(data),
        'standard_deviation': np.std(data)
    }
    return features

def ml_model(features, labels):
    # Train a simple random forest classifier for this example
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    return accuracy

# Main execution
file_path = "path_to_your_ome.tiff"
image, metadata = load_ome_tiff(file_path)
processed_image = preprocess(image)
binary_image = segment(processed_image)
convex, complexity = morphological_analysis(binary_image)

# Extract features from each image for ML (you'd loop over many images here)
features = []
labels = []  # You would also need a label for each image
features.append(extract_features(binary_image))

# Train ML model
accuracy = ml_model(features, labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Shutdown JVM
javabridge.kill_vm()


In [ ]:
## READ TIFF STACK

from tifffile import imread
stack = imread('/Users/katherineridley/APPFIRE_image/OMETIFFs/A_4_M04_OC_P+_CA1_011123_1_c0.ome.tiff', is_ome=False)


In [ ]:

from skimage.filters import gaussian
smoothed = gaussian(stack, sigma=1)


In [ ]:
from skimage.filters import threshold_otsu
thresh = threshold_otsu(smoothed)
binary = smoothed > thresh

# ALTERNATIVES TO THRESHOLDING
While thresholding is a basic technique, many advanced alternatives can yield better segmentation results, especially in challenging datasets with variability in intensity, noise, and other artifacts. Some of the more advanced alternatives to threshold-based segmentation include:

1. **Active Contours (Snake Algorithms)**:
    - Active contours aim to find an optimal curve (in 2D) or surface (in 3D) separating the object from the background.
    - The initial contour is iteratively evolved by considering image-based forces (like gradients) and internal constraints (like smoothness or elasticity).

2. **Watershed Segmentation**:
    - This method treats image intensity as a topographic landscape with hills and valleys.
    - "Floodwaters" are poured onto this landscape from predefined markers, and "basins" are filled up, resulting in segmented regions.

3. **Random Forests and Machine Learning**:
    - This approach involves training a classifier (like Random Forest) using handpicked features from labeled examples of the regions of interest and background.
    - Once the classifier is trained, it can be used to predict the class (object or background) of every pixel or voxel in the image.

4. **Deep Learning and Convolutional Neural Networks (CNNs)**:
    - Deep learning, especially CNNs, has achieved state-of-the-art performance in many segmentation tasks.
    - U-Net is a popular CNN architecture designed for biomedical image segmentation. It requires labeled training data.
    - Transfer learning, where a pre-trained model is fine-tuned on a smaller dataset, can be useful if you don't have a large annotated dataset.

5. **Level Set Methods**:
    - Level set methods represent a curve (in 2D) or a surface (in 3D) as the zero level of a higher-dimensional function.
    - This implicit representation is evolved over time according to predefined criteria, allowing for topological changes and providing a flexible framework for segmentation.

6. **Graph-based Segmentation**:
    - In this method, an image is represented as a graph, where pixels/voxels are nodes and edges are defined based on pixel/voxel relationships.
    - Graph cuts or minimum cut methods can be used to find the optimal segmentation.

7. **Region Growing**:
    - Starting from a seed point, pixels or voxels are iteratively added to the region based on predefined criteria, such as intensity similarity.

8. **Atlas-guided and Model-based Methods**:
    - For structured images (like MRIs of the brain), predefined models or atlases can guide the segmentation.
    - These methods aim to match or register the model to the observed data, benefiting from prior knowledge about the structure of interest.

When choosing an advanced method, it's essential to consider the nature of your data, the availability of labeled training data (for supervised methods), computational resources, and the specific challenges your images present (e.g., low contrast, overlapping objects). Often, combining multiple methods (ensemble methods or hybrid approaches) or post-processing steps can improve results further.

In [ ]:

from skimage.morphology import opening, closing, disk
refined = opening(binary, disk(3))


In [ ]:
from skimage.morphology import skeletonize_3d
skeleton = skeletonize_3d(refined)


In [ ]:
from skimage.measure import label, regionprops

labeled = label(binary)
properties = regionprops(binary)

for prop in properties:
    volume = prop.area
    eccentricity = prop.eccentricity
    

    
    # ... and other measurements


In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(your_confocal_data)
